In [1]:
# import intake
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pylab as plt
import matplotlib.cm as cm
import cmocean.cm as cmo
from scipy.interpolate import CloughTocher2DInterpolator, LinearNDInterpolator, NearestNDInterpolator
import gribscan
import intake
import numcodecs
import dask
import cartopy.crs as ccrs


cat = intake.open_catalog("/home/b/b381639/nextGems_Cycle2/catalog.yaml")
#run = cat.IFS["tco2559-ng5"]
run = cat.IFS["tco1279-orca025"]
data = run.ICMGG_atm2d.to_dask()

In [2]:
var='10v'
data1=data[var]#.isel({'time': slice(0,240)})

In [3]:
Varout=xr.zeros_like(data[var][0:713,:])
outdata = data1.resample(time="6H").mean(dim="time")
#https://www.statology.org/python-resample-time-series/

In [7]:
outdata.time

<xarray.DataArray 'time' (time: 2849)>
array(['2020-01-20T00:00:00.000000000', '2020-01-20T06:00:00.000000000',
       '2020-01-20T12:00:00.000000000', ..., '2021-12-31T12:00:00.000000000',
       '2021-12-31T18:00:00.000000000', '2022-01-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20 2020-01-20T06:00:00 ... 2022-01-01

### press Y in command mod and change it to code to test if the variables are accumulated


var='u10'
test=data[var][1000:2000,100000]
#[150:420,500]
test.plot()

In [8]:
#2t,2d,sst,sp,10u,10v is not accumulated
var='10v'
data1=data[var]#.isel({'time': slice(0,240)})
Varout=xr.zeros_like(data[var][0:713,:])
outdate = data1.resample(time="1D").mean(dim="time") #.compute()

Varout.data=outdate
Varout.coords['time']=outdate.time

### Save needed variables to nc file

In [9]:
Varout

<xarray.DataArray '10v' (time: 713, value: 6599680)>
dask.array<stack, shape=(713, 6599680), dtype=float64, chunksize=(1, 6599680), chunktype=numpy.ndarray>
Coordinates:
    lat      (value) float64 dask.array<chunksize=(6599680,), meta=np.ndarray>
    lon      (value) float64 dask.array<chunksize=(6599680,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-20 2020-01-21 ... 2022-01-01
Dimensions without coordinates: value
Attributes:
    paramId:                    166
    dataType:                   fc
    numberOfPoints:             6599680
    typeOfLevel:                surface
    stepUnits:                  1
    stepType:                   instant
    gridType:                   reduced_gg
    shortName:                  10v
    units:                      m s**-1
    name:                       10 metre V wind component
    cfVarName:                  v10
    missingValue:               9999
    totalNumber:                0
    NV:                         0
    gridDefinitionDescription:  Gaussian Latitude/Longitude Grid

In [10]:
Varout.to_netcdf('/work/bb1153/b381639/IFS/'+var+'_daily_tco1279-orca025.nc')

In [ ]:




#creat a output data with all the attribution needed
#Acumunated var slhf, sshf, 
#2t, 2d, sst,sp,tp,10u, 10v
var='sst'
#######################

Varout=xr.zeros_like(data[var][0:711,:])
times = pd.date_range("2020/01/20",periods=711,freq='D',closed='left')
time_da = xr.DataArray(times, [('time', times)])

#########
#Output Varout
#########
Varout.coords['time']=time_da

In [ ]:




#creat a output data with all the attribution needed
#Acumunated var slhf, sshf, 
#2t, 2d, sst,sp,tp,10u, 10v
var='sst'
#######################

Varout=xr.zeros_like(data[var][0:711,:])
times = pd.date_range("2020/01/20",periods=711,freq='D',closed='left')
time_da = xr.DataArray(times, [('time', times)])

#########
#Output Varout
#########
Varout.coords['time']=time_da

In [85]:
#But fluxes are accumulated from start of the month, or for Jan 2020, it would be Jan 20th. 
#So we need to take the difference from day after to current day. 
####VARarray has the unit from J/s to W/day

#creat a calendar array

yyyys=2020
yyyye=2021
yr=yyyys
#day=4
mms=1
mme=12
cdatearr=[]
dayarr=[]

while yr<=yyyye:
    yyyy=str(yr)
    mth=mms
    if yr==yyyye:
        mthe=mme
    else:
        mthe=12
    while mth<=mthe:
        if mth<10:
            mm='0'+str(mth)
        else:
            mm=str(mth)
        if mth==1 or mth==3 or mth==5 or mth==7 or mth==8 or mth==10 or mth==12:
            daye=31
        elif mth==4 or mth==6 or mth==9 or mth==11:
            daye=30
        if (yr%4)==0 and mth==2:
            daye=29
        elif (yr%4)!=0 and mth==2:
            daye=28
        if yr==2020 and mth==1:
            days=20
        else:
            days=1
        day=days
        tidx=0
        while day<=daye:
            if day<10:
                dd='0'+str(day)
            else:
                dd=str(day)
            cdatearr.append(yyyy+'-'+mm+'-'+dd)
            dayarr.append(day)
            #print(data.str.sel(time=yyyy+'-'+mm+'-'+dd+'T00:00:00.000000000').time)
            day=day+1
            #cidx=cidx+1
        mth=mth+1
    yr=yr+1

print(np.shape(cdatearr)[0]-1)
#print(cdatearr)

711


In [86]:
# day2-day1 = day2_daily

VARarray=[]
cidx=0
while cidx<=np.shape(cdatearr)[0]-2:
    cdate=cdatearr[cidx]
    cdatep=cdatearr[cidx+1]
    #print(dayarr[cidx+1])
    #dlen=len(dayarr[cidx+1])
    if dayarr[cidx+1]==2 or cidx==0:
        #print('Use only '+cdatep)
        #print('datestamp='+cdate+'T00:00:00.000000000')
        VARarray.append(data[var].sel(time=cdatep+'T00:00:00.000000000').reset_coords('time', drop=True) /86400)


        #print(SHF.max())
    else:
        #print('Use '+cdatep+' - '+cdate)
        VARarray.append((data[var].sel(time=cdatep+'T00:00:00.000000000')-data[var].sel(time=cdate+'T00:00:00.000000000') )/86400)   

        #SHFarray.append(SHF)
    cidx=cidx+1

Varout.data=xr.concat(VARarray,dim='time')
Varout.coords['time']=time_da
Varout.attrs['units']='W m**-2'

In [87]:
Varout.to_netcdf('/scratch/b/b381639/IFS/'+var+'_daily_tco1279-orca025.nc')